# Welcome to my new Kernel to NLP tasks

I'm starting to focus on NLP tasks and I decided to find for good challenges on Kaggle, and for luck, I discovered this competition.

I hope it can be useful to the other fellows I'm sure that I will learn a lot in this amazing world of NLP. 

Come with me and let's discover some interesting questions about this data.


# Description

Computers are really good at answering questions with single, verifiable answers. But, humans are often still better at answering questions about opinions, recommendations, or personal experiences.

Humans are better at addressing subjective questions that require a deeper, multidimensional understanding of context - something computers aren't trained to do well…yet.. Questions can take many forms - some have multi-sentence elaborations, others may be simple curiosity or a fully developed problem. They can have multiple intents, or seek advice and opinions. Some may be helpful and others interesting. Some are simple right or wrong.

<img src="https://storage.googleapis.com/kaggle-media/competitions/google-research/human_computable_dimensions_1.png" alt="QA Google">

Unfortunately, it’s hard to build better subjective question-answering algorithms because of a lack of data and predictive models. That’s why the CrowdSource team at Google Research, a group dedicated to advancing NLP and other types of ML science via crowdsourcing, has collected data on a number of these quality scoring aspects.

In this competition, you’re challenged to use this new dataset to build predictive algorithms for different subjective aspects of question-answering. The question-answer pairs were gathered from nearly 70 different websites, in a "common-sense" fashion. Our raters received minimal guidance and training, and relied largely on their subjective interpretation of the prompts. As such, each prompt was crafted in the most intuitive fashion so that raters could simply use their common-sense to complete the task. By lessening our dependency on complicated and opaque rating guidelines, we hope to increase the re-use value of this data set. What you see is what you get!

Demonstrating these subjective labels can be predicted reliably can shine a new light on this research area. Results from this competition will inform the way future intelligent Q&A systems will get built, hopefully contributing to them becoming more human-like.

# INITIAL QUESTIONS: 
- What's the intersection between users that do the question and the who don't. 
- In general (mean), how many different topics an user do questions?
- How many questions and/or answers an users usually do? 
- We can see clear patterns in different topics? (for instance, well writen in literacture topic...)
- How many different topics we have?
- The size of questions and answers are same in all categories?
- We can detect the sentiment of QA of different categories?
- What's the most frequent words by categories? 
- How many different topics we can detect in the QA topics? 
- And much more questions that probably will raise when I put my hands-on it! 



## After that we got the answers I will create some features, preprocess the data and model an algorithm that can solve this problem;

# Enjoy and stay tuned to the next updates

## Importing the needed libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib_venn import venn2, venn3, venn2_circles, venn3_circles #to create intersection graphs
import matplotlib.pyplot as plt #to plot show the charts
import seaborn as sns
from scipy import stats

from nltk import word_tokenize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Importing the data

In [ ]:
df_train = pd.read_csv("../input/google-quest-challenge/train.csv")
df_test = pd.read_csv("../input/google-quest-challenge/test.csv")
df_sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")

### Creating func to resume the dataset

In [ ]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

In [ ]:
resumetable(df_train)[:10]

Cool! It's a good start to meet our data; 
We have **6079 Answers Unique ID's** of QA Labeled. <br>
The resume also shows that **3215 different users did questions** and **4114 different users answered** questions

We can note that the Entropy of the values are very high and we don't have missing values; 




## Test dataset

In [ ]:
resumetable(df_test)[:10]

We have 476 examples in the test dataset.

## Shape of the Submission

In [ ]:
print(f"Shape of submission: {df_sub.shape}")
target_cols = df_train[df_train.columns[df_train.columns.isin(df_sub.columns[1:])]].columns

# First look in "the face"(haha) of our data

In [ ]:
df_train.head(3)

Now, we are ready to start exploring the data to try get a good insight that can help us to build the Algorithm;<br>
We can see that we have many different metrics that was included only on training data... It will be useful to we better understand the categories.

## UNIQUE QUESTIONS
- Let's start by a more generalist understanding and how the questions are distributed between the different categories and hosts

In [ ]:
## I will use the host column, split and get the first string
for i in range(len(df_train)):
    df_train.loc[i,'host_cat'] = df_train.host.str.split('.')[i][0]
df_train.drop('host', axis=1, inplace=True)

In [ ]:
host = df_train.groupby(['host_cat'])['url'].nunique().sort_values(ascending=False)
category = df_train.groupby(['category'])['url'].nunique().sort_values(ascending=False)

plt.figure(figsize=(16,12))
plt.suptitle('Unique URL by Host and Categories', size=22)

plt.subplot(211)
g0 = sns.barplot(x=category.index, y=category.values, color='blue')
g0.set_title("Unique Answers by category", fontsize=22)
g0.set_xlabel("Category Name", fontsize=19)
g0.set_ylabel("Total Count", fontsize=19)
#g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
for p in g0.patches:
    height = p.get_height()
    g0.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.1f}%'.format(height/category.sum()*100),
            ha="center",fontsize=11) 

plt.subplot(212)
g1 = sns.barplot(x=host[:20].index, y=host[:20].values, color='blue')
g1.set_title("TOP 20 HOSTS with more UNIQUE questions", fontsize=22)
g1.set_xlabel("Host Name", fontsize=19)
g1.set_ylabel("Total Count", fontsize=19)
g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
for p in g1.patches:
    height = p.get_height()
    g1.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.1f}%'.format(height/host.sum()*100),
            ha="center",fontsize=11) 
    
plt.subplots_adjust(hspace = 0.3, top = 0.90)

plt.show()

First chart:
The most common category in Stackexchange fórums are:<br>
1 - Technology(41.8%)<br>
2 - Stackoverflow (21.2%)<br>
3 - Culture (14.7%)<br>
4 - Science (11.2)<br>
5 - Life Arts(10.9)<br>
It's not so unbalanced, so it can be useful to train the model. 

Second Chart:
The most common category is the Stackoverflow with 20.6% of the total open topics(questions);<br>
After the StackOverflow we can see that only 7 categories have a ratio highest than 2%, we have in total 59;

The group of Stackoverflow, eletronics, superuser, serverfault, english, math, tex, physics and askubuntu together has **almost 46% of the total** questions. 


## Questions and Answer Unique Users and some values
As we saw in the Resume table, 
> Total Unique Users in 'Question User Name': 3215 <br>Total Unique Users in 'Answer User Name': 4114 <br>
    
Let's take a first look on the users intersection between who ask something and who try to answer. 


In [ ]:
print(f"Total Unique Users in 'Question User Name': {df_train['question_user_name'].nunique()}")
print(f"Total Unique Users in 'Answer User Name': {df_train['answer_user_name'].nunique()}")

## Ploting the Venn Diagram.

In [ ]:
plt.figure(figsize=(12,8))

venn2([set(df_train['question_user_name'].value_counts(dropna=False).index), 
       set(df_train['answer_user_name'].value_counts(dropna=False).index)],
      set_labels=('Question Users', 'Answer Users'), alpha=.5)
plt.title('Comparison of Question and Answer Users Intersection\n', fontsize=20)

plt.show()

It's very interesting. <br>
We can see that a small part of total users has done both actions on the platform. <br>
I thought it had a more balanced ratio in users that do questions and who answer it;



##  QA USERS Intersection by CATEGORY feature

In [ ]:
import matplotlib.gridspec as gridspec # to do the grid of plots

grid = gridspec.GridSpec(3, 3)
plt.figure(figsize=(16,3*4))

plt.suptitle('Intersection QA USERS \nQuestions and Answers by different CATEGORIES', size=20)

for n, col in enumerate(df_train['category'].value_counts().index):
    ax = plt.subplot(grid[n])
    venn2([set(df_train[df_train.category == col]['question_user_name'].value_counts(dropna=False).index), 
           set(df_train[df_train.category == col]['answer_user_name'].value_counts(dropna=False).index)],
      set_labels=('Question Users', 'Answer Users'), )
    ax.set_title(str(col), fontsize=15)
    ax.set_xlabel('')
    #plt.subplots_adjust(top = 0.98, wspace=.9, hspace=.9)
    
plt.subplots_adjust(top = 0.9, hspace=.1)

plt.show()

It's interesting, we can see some difference in the intersection ratio of the USERS in the different categories;<br>
Take a look on Stackoverflow, it has a big difference between people who ask and people who answer questions; 

## QA USERS Intersection by Host feature

In [ ]:
grid = gridspec.GridSpec(5, 3)
plt.figure(figsize=(16,4.5*4))

plt.suptitle('Intersection QA USERS - TOP 15 \nQuestions and Answers by different HOSTS', size=20)
top_host = df_train['host_cat'].value_counts()[:15].index
for n, col in enumerate(top_host):
    ax = plt.subplot(grid[n])
    venn2([set(df_train[df_train.host_cat == col]['question_user_name'].value_counts(dropna=False).index), 
           set(df_train[df_train.host_cat == col]['answer_user_name'].value_counts(dropna=False).index)],
      set_labels=('Question Users', 'Answer Users'), )
    ax.set_title(str(col), fontsize=15)
    ax.set_xlabel('')
    #plt.subplots_adjust(top = 0.98, wspace=.9, hspace=.9)
    
plt.subplots_adjust(top = 0.9, hspace=.1)

plt.show()

Cool! I find it very insightful hahah<br>
It's interesting to note that programmers host has almost 1:3 ratio between QA Users, and only one in the intersection.<br>
Only for curiosity, we can take a look on one some of the intersection users to see what type of questions was done by them

# Title and Body Lenghts of questions 
- I will start counting the Number of words in each question and set it to a new column
- After I will plot some distributions about the titles and the bodies of the questions
- The idea is to see te difference by categories

In [ ]:
# Tokenize each item in the review column
word_tokens = [word_tokenize(question) for question in df_train.question_body]

# Create an empty list to store the length of the reviews
len_tokens = []

# Iterate over the word_tokens list and determine the length of each item
for i in range(len(word_tokens)):
     len_tokens.append(len(word_tokens[i]))

# Create a new feature for the lengh of each review
df_train['question_n_words'] = len_tokens

In [ ]:
grid = gridspec.GridSpec(5, 3)
plt.figure(figsize=(16,6*4))

plt.suptitle('Title and Question Lenghts by Different Categories \nThe Mean in RED - Also 5% and 95% lines', size=20)
count=0
top_cats=df_train['category'].value_counts().index
for n, col in enumerate(top_cats):
    for i, q_t in enumerate(['question_title', 'question_body', 'question_n_words']):
        ax = plt.subplot(grid[count])
        if q_t == 'question_n_words':
            sns.distplot(df_train[df_train['category'] == col][q_t], bins = 50, 
                         color='g', label="RED - 50%") 
            ax.set_title(f"Distribution of {str(col)} \nQuestion #Total Words Distribution", fontsize=15)
            ax.axvline(df_train[df_train['category'] == col][q_t].quantile(.95))
            ax.axvline(df_train[df_train['category'] == col][q_t].quantile(.05))
            mean_val = df_train[df_train['category'] == col][q_t].mean()
            ax.axvline(mean_val, color='red' )
            ax.set_xlabel('')            
        else:
            sns.distplot(df_train[df_train['category'] == col][q_t].str.len(), bins = 50, 
                         color='g', label="RED - 50%") 
            ax.set_title(f"Distribution of {str(col)} \n{str(q_t)}", fontsize=15)
            ax.axvline(df_train[df_train['category'] == col][q_t].str.len().quantile(.95))
            ax.axvline(df_train[df_train['category'] == col][q_t].str.len().quantile(.05))
            mean_val = df_train[df_train['category'] == col][q_t].str.len().mean()
            ax.axvline(mean_val, color='red' )
            #ax.text(x=mean_val*1.1, y=.02, s='Holiday in US', alpha=0.7, color='#334f8d')
            ax.set_xlabel('')
        count+=1
        
plt.subplots_adjust(top = 0.90, hspace=.4, wspace=.15)
plt.show()

Cool! <br>
The quantiles of the distributions are very similar in all categories in both title and question lenghts.



# Exploring Target Features
- Let's explore the target features to find some clear pattern that can be useful.
- I will apply some techniques of dimensionality reduction and verify the explained variability and f

## Distribution of all Target Features


In [ ]:
grid = gridspec.GridSpec(10, 3)

plt.figure(figsize=(16,8*4))
count=0
plt.suptitle('Distribution of QA metrics (Target Features)', size=20)
# top_host = df_train['host_cat'].value_counts()[:15].index
for n, col in enumerate(target_cols):
    #if df_train[target_cols].std()[col] > .15:
    ax = plt.subplot(grid[count])
    sns.boxplot(x='category', y=col, data=df_train)
    ax.set_title(str(col), fontsize=13)
    ax.set_xlabel('')
    ax.set_ylabel(' ')
    count+=1
    ax.set_xticklabels(ax.get_xticklabels(),rotation=45)

plt.subplots_adjust(top = 0.95, hspace=.9, wspace=.2)

plt.show()



Cool. It's a informative chart where we can get the difference between the categories to each target feature.
- I think that worth to spent a time to analyze it in depth
For instance: 
- "question body critical" has an interesting distribution between the different categories. 
- "question well writen" shows that Life Arts and Culture has highest quality in the written. 
- And many other distributions that can be meaninful.

Also, we have some features taht we can't see a clear difference between the categories. 

Let's try to extract the PCA of all this features and see what it can shows to us. 

## Principal Component Analyzis - N components

In [ ]:
pca = PCA(n_components=3, random_state=42)

principalComponents = pca.fit_transform(df_train[target_cols])

principalDf = pd.DataFrame(principalComponents)

# df.drop(cols, axis=1, inplace=True)
prefix='Target_PCA'
principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

df_train = pd.concat([df_train, principalDf], axis=1)

In [ ]:
print("TOP 3 PCA Explanability: ")
[print(str(f"{i+1} - {round(pca*100,3)}%")) for i, pca in enumerate(pca.explained_variance_ratio_[:3])]
print(f"Sum of 3 Principal components: {round(pca.explained_variance_ratio_[:3].sum()*100,3)}%")

It's not so much, but it's a important value that means that we have some different patterns. 
- Let's create some charts to explain

# Ploting PCA of Target
- ploting the first and second principal components

In [ ]:
plt.figure(figsize=(15,6))
g = sns.scatterplot(x='Target_PCA0', y='Target_PCA1', data=df_train, hue='category')
g.set_title("PCA Components Distribution by Categories", fontsize=22)
g.set_xlabel("TARGET PCA 0", fontsize=16)
g.set_ylabel("TARGET PCA 1", fontsize=16)

plt.show()

Cool!!! It shows us that the questions in Technology and Stackoverflow has different pattern than the culture, science, life arts, what make a lot of sense. <br>

Below, I will take a look in the difference between the hosts. 

# PCA of target BY HOSTS
ploting pca to the top 15 hosts with more unique questions

In [ ]:
g = sns.FacetGrid(df_train[df_train.host_cat.isin(top_host)], col='host_cat',
                  col_wrap=3, height=3, aspect=1.5, hue='category')

g.map(sns.scatterplot, "Target_PCA0", "Target_PCA1", alpha=.5 ).add_legend();
g.set_titles('{col_name}', fontsize=17)
plt.show()

GREAT! We can clearly note different patterns in the target PCA. 
For instance, take a look on askubuntu, stackoverflow, tex and culture, English, physics and so on... It's a insightful information.

# Finding for Correlation in Target Features

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_train[target_cols].corr(),vmin=-1,cmap='YlGnBu')

We can note that some features are high correlated.<br>
For instance: Critical questions and questions interestingness self seems more well written. 

# Questions and Answers Texts
Let's start ploting some WordClouds by Categories. 


In [ ]:
from wordcloud import WordCloud, STOPWORDS

stopwords = set(STOPWORDS)
newStopWords = ['amp', 'gt', 'lt', 'div', 'id',
                'fi', 'will', 'use', 'one', 'nbsp', 'need']
stopwords.update(newStopWords)

In [ ]:
grid = gridspec.GridSpec(5, 2)

plt.figure(figsize=(16,7*4))

plt.suptitle('Word Cloud OF CATEGORY FEATURE', size=20)

for n, col in enumerate(df_train['category'].value_counts().index):
    ax = plt.subplot(grid[n])  
    
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stopwords,
        max_words=250,
        max_font_size=100, 
        width=400, height=280,
        random_state=42,
    ).generate(" ".join(df_train[df_train['category'] == col]['answer'].astype(str)))

    #print(wordcloud)

    plt.imshow(wordcloud)
    plt.title(f"Category: {col}",fontsize=18)
    plt.axis('off')
plt.subplots_adjust(top = 0.95, hspace=.2, wspace=.1 )

plt.show()

Nice I can see some interesting words in the different categories, but only stackoverflow seems more clear. Let's explore it in further;

# WordCloud by HOSTS
- TOP 10 hosts with more unique questions

In [ ]:
#newStopWords = ['fruit', "Drink", "black"]

#stopwords.update(newStopWords)

import matplotlib.gridspec as gridspec # to do the grid of plots
grid = gridspec.GridSpec(5, 2)

plt.figure(figsize=(16,7*4))

plt.suptitle('Answers Word Cloud \nTOP 10 hosts with more questions', size=20)

for n, col in enumerate(df_train['host_cat'].value_counts()[:10].index):
    ax = plt.subplot(grid[n])   
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stopwords,
        max_words=250,
        max_font_size=100, 
        width=400, height=280,
        random_state=42,
    ).generate(" ".join(df_train[df_train['host_cat'] == col]['answer'].astype(str)))

    #print(wordcloud)

    plt.imshow(wordcloud)
    plt.title(f"Host: {col}",fontsize=18)
    plt.axis('off')
    
plt.subplots_adjust(top = 0.95, hspace=.2, wspace=.1 )

plt.show()

Cool! 
The wordcloud of the hosts are very meaningful of the different communities

# Answers Sentiment analysis 

In [ ]:
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df_train['ans_polarity']= df_train['answer'].apply(pol)
df_train['ans_subjectivity']= df_train['answer'].apply(sub)

df_train[['answer', 'category', 'ans_polarity', 'ans_subjectivity']].head()

Cool! Now that we got some metrics about the posible sentiment of the answers, let get some quantitative analysis of it.

## Ploting the polarity x the subjetictivy

In [ ]:
plt.figure(figsize=(16,5))

g = sns.scatterplot(x='ans_polarity', y='ans_subjectivity', 
                    data=df_train, hue='category')
g.set_title("Sentiment Analyzis (Polarity x Subjectivity) by 'Category' Feature", fontsize=21)
g.set_xlabel("Polarity distribution",fontsize=18)
g.set_ylabel("Subjective ",fontsize=18)

plt.show()

We can't see a clear difference in the answers. But seems that the answers tends to have a more positive value in polarity <br>
Let's find other way to access this informations in a better and more objective way.

## Getting some statistics of Polarity and Subjectivity by the Category feature

In [ ]:
polarity_answers = df_train.groupby('category')['ans_polarity', 'ans_subjectivity'].describe().reset_index()

polarity_answers

De degree of subjectivity of the Stackoverflow is smaller than the other categories. <br>
We can also note a smaller level of polarity in the answer. In opposite of it, Life Arts has the highest polarity and subjectivity what make sense.

# Trigrams by TOP 10 HOSTS

In [ ]:
stopwords.update(['amp', 'lt', 'gt', 'frac'])

In [ ]:
import re
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not",
                    "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
                    "he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", 
                    "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have",
                    "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", 
                    "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                    "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", 
                    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
                    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                    "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                    "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                    "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
                    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", 
                    "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", 
                    "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
                    "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", 
                    "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", 
                    "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
                    "y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                    "you'll've": "you will have", "you're":"you are", 'you re':"you are",'youre': "you are", "you've": "you have", }

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>',
          '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£','·', '_', '{', '}', '©', '^', '®', '`', 
          '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
          '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
          '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',  '▒', '：', '¼', '⊕', '▼', '▪', '†', '■',
          '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
          '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬',
          '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', #'lt', 'gt', 'amp', 'div', 'ex', 'le', 'http', 'www', 'vo', '\n'
         ]

def clean_text(x):
    x = str(x)
    
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


## Random Example of Questions + Answers

In [ ]:
# Here, the order is important
df_train['answer'] = df_train['answer'].apply(replace_contractions)
df_train['answer'] = df_train['answer'].apply(clean_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

grid = gridspec.GridSpec(5, 3)
plt.figure(figsize=(16,6*4))

for n, cat in enumerate(top_host[:9]):
    
    ax = plt.subplot(grid[n])   
    # print(f'PRINCIPAL WORDS CATEGORY: {cat}')
    # vectorizer = CountVectorizer(ngram_range = (3,3)) 
    # X1 = vectorizer.fit_transform(df_train[df_train['host_cat'] == cat]['answer'])  
 
    min_df_val = round(len(df_train[df_train['host_cat'] == cat]) - len(df_train[df_train['host_cat'] == cat]) * .99)
    max_df_val = round(len(df_train[df_train['host_cat'] == cat]) - len(df_train[df_train['host_cat'] == cat]) * .3)
    
    # Applying TFIDF 
    vectorizer = TfidfVectorizer(ngram_range = (2,2), min_df=5, stop_words='english',
                                 max_df=.5) 
    X2 = vectorizer.fit_transform(df_train[df_train['host_cat'] == cat]['answer']) 
    features = (vectorizer.get_feature_names()) 
    scores = (X2.toarray()) 

    # Getting top ranking features 
    sums = X2.sum(axis = 0) 
    data1 = [] 
    
    for col, term in enumerate(features): 
        data1.append( (term, sums[0,col] )) 

    ranking = pd.DataFrame(data1, columns = ['term','rank']) 
    words = (ranking.sort_values('rank', ascending = False))[:10]
    
    sns.barplot(x='term', y='rank', data=words, ax=ax, 
                color='blue', orient='v')
    ax.set_title(f"Top rank Trigram of: {cat}")
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    ax.set_ylabel(' ')
    ax.set_xlabel(" ")

plt.subplots_adjust(top = 0.95, hspace=.9, wspace=.1)

plt.show()

It needs a more appropriate text preprocessing but it give us a good first understand about the words in different categories

# Start Modellling
- Preprocessing
- Implement a simple solution BERT using TfIdf as baseline 

> I will implement the amazing pythonic preprocessing of @pavelvpster kernel, you can access <a href="https://www.kaggle.com/pavelvpster/google-q-a-labeling-tf-idf-pytorch">here</a>

## Preprocessing

In [ ]:
extra_cols = ['host_cat', 'question_n_words', 'Target_PCA0',
              'Target_PCA1', 'Target_PCA2', 'ans_polarity', 'ans_subjectivity']

Setting the X and y

In [ ]:
y_train = df_train[target_cols].copy()
X_train = df_train.drop(list(extra_cols) + list(target_cols), axis=1)
del df_train

X_test = df_test.copy()
del df_test

# Start Modeling
- I will start by a simple model to use as baseline. As it's my first time with a QA task, I searched for good solutions. 
- The QA model implementation below, I saw in @hamditarek amazing kernel about LDA and LSA topic modelling. Kernel Link below on the resources links; 

Version 2: <br>
I'm now testing the solutions of akensert

In [ ]:
## Shell 
!pip install ../input/sacremoses > /dev/null

import sys
sys.path.insert(0, "../input/transformers/")


# Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

from transformers import *

np.set_printoptions(suppress=True)
print(tf.__version__)

## Setting thePaths to the pre trained models and 

In [ ]:
np.set_printoptions(suppress=True)

from transformers import *

BERT_PATH = '../input/bert-base-uncased-huggingface-transformer/'
tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-uncased-vocab.txt')

MAX_SEQUENCE_LENGTH = 512


## Functions to preprocess the data to BERT archtecture

In [ ]:
## The function to creat the masks using to the title, question and answer
def _convert_to_transformer_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids_q, input_masks_q, input_segments_q = return_id(
        title + ' ' + question, None, 'longest_first', max_sequence_length)
    
    input_ids_a, input_masks_a, input_segments_a = return_id(
        answer, None, 'longest_first', max_sequence_length)
    
    return [input_ids_q, input_masks_q, input_segments_q,
            input_ids_a, input_masks_a, input_segments_a]

# Computing the inputs
def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    
    input_ids_q, input_masks_q, input_segments_q = [], [], []
    input_ids_a, input_masks_a, input_segments_a = [], [], []
    
    for _, instance in tqdm(df[columns].iterrows()):
        
        t, q, a = instance.question_title, instance.question_body, instance.answer

        ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = \
        _convert_to_transformer_inputs(t, q, a, tokenizer, max_sequence_length)
        
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)

        input_ids_a.append(ids_a)
        input_masks_a.append(masks_a)
        input_segments_a.append(segments_a)
        
    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32),
            np.asarray(input_ids_a, dtype=np.int32), 
            np.asarray(input_masks_a, dtype=np.int32), 
            np.asarray(input_segments_a, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

## Creating the model

In [ ]:

## Computing the error metric to the model optimization
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)

def create_model():
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    # normally ".from_pretrained('bert-base-uncased')", but because of no internet, the 
    # pretrained model has been downloaded manually and uploaded to kaggle. 
    bert_model = TFBertModel.from_pretrained(
        BERT_PATH+'bert-base-uncased-tf_model.h5', config=config)
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
    x = tf.keras.layers.Concatenate()([q, a])
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(30, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn,], outputs=x)
    
    return model

## Processing the inputs and outputs

In [ ]:
outputs = compute_output_arrays(y_train, y_train.columns)
inputs = compute_input_arrays(X_train, X_train.columns, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(X_test, X_test.columns, tokenizer, MAX_SEQUENCE_LENGTH)

## Trainning the model

In [ ]:
## Creating Kfold with 5 splits 
gkf = GroupKFold(n_splits=5).split(X=X_train.question_body, groups=X_train.question_body)

## to receive predictions
valid_preds = []
test_preds = []

## Looping throught the folds
for fold, (train_idx, valid_idx) in enumerate(gkf):
    
    # will actually only do 2 folds (out of 5) to manage < 2h
    if fold in [0 , 2, 4]:
        
        ## Train index from Kfold 
        train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
        train_outputs = outputs[train_idx]
        ## Valid index from Kfold 
        valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
        valid_outputs = outputs[valid_idx]
        
        K.clear_session()
        
        ## Instantiating the Bert Model
        model = create_model()
        optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
        model.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        ## Fiting the model
        model.fit(train_inputs, train_outputs, epochs=2, batch_size=6)
        
        # model.save_weights(f'bert-{fold}.h5')
        valid_preds.append(model.predict(valid_inputs))
        # predicting the test set and appending to test_preds
        test_preds.append(model.predict(test_inputs))
        
        # Calculating the error in the valid set
        rho_val = compute_spearmanr_ignore_nan(valid_outputs, valid_preds[-1])
        print('validation score = ', rho_val)

## Dataframe to CSV 
- Saving the prediction file in csv extension

In [ ]:
df_sub.iloc[:, 1:] = np.average(test_preds, axis=0) # for weighted average set weights=[...]

df_sub.to_csv('submission.csv', index=False)

# NOTE: THIS KERNEL IS NOT FINISHED

### Stay tuned and if this kernel was useful for you, please upvote the kernel

Some of sources/references that I used in this kernel:<br>
https://www.kaggle.com/abhinand05/bert-for-humans-tutorial-baseline-version-2 <br>
https://www.kaggle.com/pavelvpster/google-q-a-labeling-tf-idf-pytorch <br>
https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools <br>
https://mlwhiz.com/blog/2019/01/17/deeplearning_nlp_preprocess/ <br>
https://www.kaggle.com/hamditarek/get-started-with-nlp-lda-lsa <br>
https://www.kaggle.com/akensert/bert-base-tf2-0-now-huggingface-transformer<br><br>
The books:<br>
- Natural Language Processing and Computational Linguistics. A practical Guide to Text Analysis with Python, Gensim, spaCy and Keras (Packt-2018) - Bhargav Srinivasa<br>
- Python Natural Language Processing (Packt-2017) - Jalaj Thanaki